In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from scipy.special import rel_entr
from scipy.stats import entropy
import scipy.stats
import math
import warnings 
warnings.filterwarnings("ignore") 

from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Tue Jun 28 16:21:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 27%   35C    P8    22W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 26%   

# MongoDB

In [4]:
from pymongo import MongoClient
import pymongo

In [5]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_cn_2011 = db.rm_patent_cn_2011
    db_rm_us_2011 = db.rm_patent_us_2011
    db_rm_de_2011 = db.rm_patent_de_2011

    
except errors.ConnectionFailure as err:
    print(err)

# 2011

## Company數量

In [6]:
data_assignee = db_rm_de_2011.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [7]:
df_family_de = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [8]:
data_assignee = db_rm_cn_2011.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_cn = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [9]:
data_assignee = db_rm_us_2011.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_us = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [10]:
# final_df = pd.concat([df_family_de,df_family_cn,df_family_us],axis=0)

In [11]:
# 取總數前15家公司
df_family_us.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(15)

,company,count
19,Qualcomm Inc,14
28,Xerox Corp,13
12,LG Electronics Inc,10
21,Samsung Electronics Co Ltd,7
11,IoT Holdings Inc,6
25,Telefonaktiebolaget LM Ericsson AB,6
6,Fujifilm Business Innovation Corp,5
29,ZTE Corp,4
4,Cisco Technology Inc,4
1,Apple Inc,3


In [12]:
com_df = df_family_us.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(15)
com_df.reset_index(drop=True, inplace=True)

In [13]:
com_df["count_log"] = com_df["count"].apply(lambda x: np.log(x))
com_df

,company,count,count_log
0,Qualcomm Inc,14,2.639057
1,Xerox Corp,13,2.564949
2,LG Electronics Inc,10,2.302585
3,Samsung Electronics Co Ltd,7,1.945910
4,IoT Holdings Inc,6,1.791759
5,Telefonaktiebolaget LM Ericsson AB,6,1.791759
6,Fujifilm Business Innovation Corp,5,1.609438
7,ZTE Corp,4,1.386294
8,Cisco Technology Inc,4,1.386294
9,Apple Inc,3,1.098612


公司名稱正規化

In [14]:
com_df["company"] = com_df.company.apply(lambda x: x.replace(" Inc","")\
                                                    .replace(" Co Ltd","")\
                                                    .replace(" Co. Ltd.","")\
                                                    .replace(" Co., Ltd.","")\
                                                    .replace(" Ltd","")\
                                                    .replace(" Corp","")\
                                                    .replace("..","")\
                                                    .replace("International Business Machines","IBM")\
                                                    .replace("Nippon Telegraph and Telephone","Nippon Telegraph & Tel")\
                                                    .replace("Alibaba Group Holding Ltd","Alibaba")\
                                                    .replace("ZTE Intelligent IoT Technology","ZTE")\
                                                    .replace("AT&T Intellectual Property I LP","AT&T")\
                                                    .replace("Microsoft Technology Licensing LLC","Microsoft")\
                                                    .replace("Telefonaktiebolaget LM Ericsson AB","Ericsson")\
                                                    .replace("Cisco Technology","Cisco Systems")\
                                                    .replace("Verizon Patent and Licensing","Verizon Communications")\
                                                    .replace("Tencent Technology Shenzhen","Tencent Holdings")\
                                                    .replace("China Mobile Communications Group","China Mobile")\
                                                    .replace("Alipay Hangzhou Information Technology","Alibaba Group")\
                                                    .replace("Beijing Xiaomi Mobile Software","Xiaomi")\
                                                    .replace("Nokia Technologies Oy","Nokia")\
                                                    .replace("Amazon Technologies","Amazon.com")\
                                                    .replace("Fujifilm Business Innovation","Fujifilm Holdings"))

In [15]:
com_df

,company,count,count_log
0,Qualcomm,14,2.639057
1,Xerox,13,2.564949
2,LG Electronics,10,2.302585
3,Samsung Electronics,7,1.945910
4,IoT Holdings,6,1.791759
5,Ericsson,6,1.791759
6,Fujifilm Holdings,5,1.609438
7,ZTE,4,1.386294
8,Cisco Systems,4,1.386294
9,Apple,3,1.098612


In [16]:
com_ls = list(com_df.company)

In [17]:
com_ls.remove('Samsung Electronics')

# Data 2011

富比士2000資料

In [18]:
f_data = pd.read_csv("../../Forbes/Forbes_2011.csv")

In [19]:
f_data[f_data["Industry"]=="Semiconductors"]

,Company,Industry,Country,Market Value,Sales,Profits,Assets,Rank,Forbes Webpage,Profits as % of Assets,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
32,Samsung Electronics,Semiconductors,South Korea,112.9,133.800,13.7000,119.3,33,http://www.forbes.com/companies/samsung-electr...,0.114837,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,Intel,Semiconductors,United States,114.5,43.600,11.5000,63.2,93,http://www.forbes.com/companies/intel,0.181962,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272,Taiwan Semiconductor,Semiconductors,Taiwan,61.3,13.300,5.1000,23.7,273,http://www.forbes.com/companies/taiwan-semicon...,0.215190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395,Texas Instruments,Semiconductors,United States,40.3,14.000,3.2000,13.4,396,http://www.forbes.com/companies/texas-instruments,0.238806,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
547,Applied Materials,Semiconductors,United States,19.8,10.400,1.4000,11.3,548,http://www.forbes.com/companies/applied-materials,0.123894,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
602,Micron Technology,Semiconductors,United States,10.2,9.000,1.8000,14.6,603,http://www.forbes.com/companies/micron-technology,0.123288,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
658,STMicroelectronics,Semiconductors,Switzerland,11.0,10.500,0.8417,13.0,658,http://www.forbes.com/companies/stmicroelectro...,0.064746,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
735,Broadcom,Semiconductors,United States,21.6,6.800,1.1000,7.9,736,http://www.forbes.com/companies/broadcom,0.139241,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
751,ASML Holding,Semiconductors,Netherlands,17.8,6.000,1.4000,8.2,752,http://www.forbes.com/companies/asml-holding,0.170732,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1030,Hynix Semiconductor,Semiconductors,South Korea,14.5,6.800,-0.2984,13.8,1031,http://www.forbes.com/companies/hynix-semicond...,-0.021623,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Forbes list  
- 同Industry：1
- 同Sector：2
- 有出現在company list：3
- Company list上有但Forbes沒有：4

In [20]:
forb_ls =[]

for i in com_ls:
    if (f_data["Company"]==i).any():
        if (f_data[f_data.Company==i]["Industry"]=="Semiconductors").any():
            forb_ls.append(1)
        # elif (f_data[f_data.Company==i]["Sector"]=="Information Technology").any():
        #     forb_ls.append(2)
        else:
            forb_ls.append(3)
    else:
        forb_ls.append(4)
    

In [21]:
forb_ls

[3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4]

## Company list competitor

In [22]:
competitor_ls=[2, 3, 2, 2, 2, 1, 2, 2, 2, 1, 4, 3, 4, 3]

## 計算NMI

In [23]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.3293382872379297
ari:  0.2125


## 公司名稱和類別

In [24]:
competitor_df = pd.DataFrame((zip(com_ls, competitor_ls, forb_ls)), columns = ['company', '2011_competitor', '2011_forb'])

In [25]:
competitor_df

,company,2011_competitor,2011_forb
0,Qualcomm,2,3
1,Xerox,3,3
2,LG Electronics,2,3
3,IoT Holdings,2,4
4,Ericsson,2,3
5,Fujifilm Holdings,1,3
6,ZTE,2,3
7,Cisco Systems,2,3
8,Apple,2,3
9,IBM,1,3


In [26]:
competitor_df.to_csv("../competitor_df/cpc_us_2011.csv", index=False)